In [1]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from flask_babel import get_locale, gettext


import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np

df_fb = pd.read_csv('../static/csv/facebook/movement_countries.csv')

df_fb.rename(columns = {'ds':'date'}, inplace = True)


eu_countries = ['austria','belgium' ,'croatia' ,'czech republic' ,'denmark',
 'estonia' ,'finland' ,'france' ,'germany', 'greece', 'iceland', 'ireland',
 'italy','lithuania' ,'luxembourg', 'netherlands', 'norway',
 'romania', 'serbia', 'spain', 'sweden', 'switzerland',
 'united kingdom', 'portugal', 'slovenia',  'hungary', 'poland',
 'bosnia and herzegovina', 'slovakia', 'bulgaria']


# filter eu countries
df_fb = df_fb[df_fb['end_name'].isin(eu_countries)]
df_fb = df_fb[df_fb['start_name'].isin(eu_countries)]
countries = sorted(df_fb.end_name.unique())

df_in = df_fb.groupby(['date', 'end_name']).agg({'travel_counts': 'sum'}).reset_index()



df_out = df_fb.groupby(['date', 'start_name']).agg({'travel_counts': 'sum'}).reset_index()


df_owid = pd.read_csv('../static/csv/owid.csv')
print(list(df_owid[:5]))
df_owid['location'] = df_owid['location'].str.lower()
df_owid = df_owid[df_owid['location'].isin(eu_countries)]
df_owid = df_owid[['date','location','new_cases_smoothed_per_million',
                   'continent','population',
                   'new_cases_smoothed','reproduction_rate','stringency_index']]


# combine facebook and owid data-set
df = pd.merge(df_fb, df_owid, how='left', left_on=['date','start_name'], 
              right_on=['date','location']).drop(columns= ['location']) 
df.rename(columns = {'new_cases_smoothed_per_million':'start_new_cases_smoothed_per_million',
                     'reproduction_rate':'start_reproduction_rate',
                    'population':'start_population','new_cases_smoothed':'start_new_cases_smoothed'}, inplace = True)
df = pd.merge(df, df_owid, how='left', 
              left_on=['date','end_name','continent'], 
              right_on=['date','location','continent']).drop(columns= ['location']) 

df.rename(columns = {'new_cases_smoothed_per_million':'end_new_cases_smoothed_per_million',
                     'population':'end_population',
                     'new_cases_smoothed':'end_new_cases_smoothed',
                     'reproduction_rate':'end_reproduction_rate'}, inplace = True)


# replace some non existing values or negative values
df['end_reproduction_rate'] = df['end_reproduction_rate'].fillna(0)
df['end_new_cases_smoothed'] = df['end_new_cases_smoothed'].fillna(0)
df['end_new_cases_smoothed_per_million'] = df['end_new_cases_smoothed_per_million'].fillna(0)
df['start_reproduction_rate'] = df['start_reproduction_rate'].fillna(0)
df['start_new_cases_smoothed'] = df['start_new_cases_smoothed'].fillna(0)
df['start_new_cases_smoothed_per_million'] = df['start_new_cases_smoothed_per_million'].fillna(0)

df['end_new_cases_smoothed'] = df['end_new_cases_smoothed'].clip(lower=0)
df['end_new_cases_smoothed_per_million'] = df['end_new_cases_smoothed_per_million'].clip(lower=0)



# compute covid flow (presumed number of incoming covid cases)
df['covid_flow'] = df['start_new_cases_smoothed_per_million'] * df['travel_counts'] / 1000000



df_in = df.groupby(['date', 'end_name', 'end_population', 'end_new_cases_smoothed',
                 'end_reproduction_rate','end_new_cases_smoothed_per_million']).agg(
    {'covid_flow': 'sum', 'travel_counts': 'sum'}).reset_index()


df_tot_travel_in = df_in.groupby(['date']).agg({'travel_counts':'sum','end_new_cases_smoothed':'sum'}).reset_index()



df_out = df.groupby(['date', 'start_name', 'start_population', 'start_new_cases_smoothed',
                 'start_reproduction_rate','start_new_cases_smoothed_per_million']).agg(
    {'covid_flow': 'sum', 'travel_counts': 'sum'}).reset_index()



df_tot_travel_out = df_out.groupby(['date']).agg({'travel_counts':'sum','start_new_cases_smoothed':'sum'}).reset_index()


countries = df_in.end_name.unique()



['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million', 'new_cases_per_million', 'new_cases_smoothed_per_million', 'total_deaths_per_million', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients', 'icu_patients_per_million', 'hosp_patients', 'hosp_patients_per_million', 'weekly_icu_admissions', 'weekly_icu_admissions_per_million', 'weekly_hosp_admissions', 'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests', 'total_tests_per_thousand', 'new_tests_per_thousand', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'positive_rate', 'tests_per_case', 'tests_units', 'stringency_index', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', '

In [21]:
def plot_in_out():
    plots = []
    
    for c in countries:
        dfc_out = df_out[df_out.start_name == c]
        dfc_in = df_in[df_in.end_name == c]
        

        #maxv = np.nanmax(dfc_in.travel_counts.rolling(7).mean().values)


        plot = go.Scatter(x=dfc_in.travel_counts.rolling(7).mean(), 
                          y=dfc_out.travel_counts.rolling(7).mean(), 
                          name=c,mode='markers')
        plots.append(plot)

    
    fig = go.Figure(data=plots, layout=go.Layout(barmode='group'))

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))
    fig.update_xaxes(type="log")
    fig.update_yaxes(type="log")
    fig.update_layout(
        height=600,
        title="In Out Analysis",
        xaxis_title="total in flow",
        yaxis_title="total out flow"
    )
    return fig

plot_in_out().show()

In [22]:
def plot_tot_travel():
    
    #layout = go.Layout(xaxis =  { 'showgrid': False},yaxis = { 'showgrid': True })
    
    #'layout': layout 
    
    fig = make_subplots(specs=[[{"secondary_y": True,}]],shared_yaxes='all',shared_xaxes='all') 
    
    maxvf = max(df_tot_travel_in.travel_counts.rolling(7).mean().values[7:])
    maxvc = max(df_tot_travel_in.end_new_cases_smoothed.rolling(7).mean().values[7:])

    
    # Add traces
    fig.add_trace(
        go.Scatter(x=df_tot_travel_in.date.values,y=df_tot_travel_in.travel_counts.rolling(7).mean(),
                   name="travel",marker_color = "red"),
        secondary_y=False,
    )

    # Add traces
    fig.add_trace(
        go.Scatter(x=df_tot_travel_in.date.values,y=df_tot_travel_in.end_new_cases_smoothed.rolling(7).mean(), 
                   name="cases",marker_color = "blue",line={'dash': 'dash'}),
        secondary_y=True,
    )
    
    
    
    #end_new_cases_smoothed

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))
    
        # Set x-axis title
    fig.update_xaxes(title_text="date")

    # Set y-axes titles
    fig.update_yaxes(title_text="Total daily flow of FB users", secondary_y=False)
    fig.update_yaxes(title_text="Total covid cases", secondary_y=True)

    
    #tickvals = [i for i in range(0,int(maxvf),500000)]
        
    
    fig.update_layout({"yaxis1": dict(range = [0, maxvf],showgrid=False),"yaxis2": dict(range = [0, maxvc],showgrid=False)})

    
    fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)


    fig.update_layout(
        height=600,
        title="Flow of users and Covid cases in Europe",
    )
    return fig

plot_tot_travel()

In [23]:
def heatmap(start_date,end_date):
    

    df_fb_ = df_fb[df_fb.date <= end_date]
    df_fb_ = df_fb_[df_fb_.date >= start_date]
    
    n = len(eu_countries)
    range_eu = range(len(eu_countries))
    mig = [[None for i in range_eu] for j in range_eu]

    import math
    df_mean_travels = df_fb_.groupby(['start_name', 'end_name']).agg({'travel_counts': 'mean'}).reset_index()
    dict_travels = df_mean_travels .set_index(['start_name','end_name']).to_dict(orient='dict')['travel_counts']
    for i in range_eu:
        for j in range_eu:
            c1 = eu_countries[i]
            c2 = eu_countries[j]
            if (c1,c2) in dict_travels.keys():
                flow_c1_c2 = dict_travels[(c1, c2)]
                mig[i][j] = math.log(flow_c1_c2)


    import plotly.graph_objects as go

    fig = go.Figure(data=go.Heatmap(
                       z=mig,
                       x=eu_countries,
                       y=eu_countries,
                       hoverongaps = False,zmin=7,zmax=11))
    fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)

    fig.update_layout(template="plotly_white", height=700, margin=dict(l=0, r=0, t=40, b=0),
                          title=gettext("Log avg flow "+start_date+"-"+end_date))
    fig.show()
    
#heatmap('2020-03-07','2020-03-13')
#heatmap('2020-04-07','2020-04-13')
#heatmap('2020-08-04','2020-08-11')
heatmap('2020-11-15','2020-11-22')

In [5]:

def plot_covid_import_index():
    plots = []

    for c in eu_countries:
        dfc = df_in[df_in.end_name == c]
        index = 100*dfc['covid_flow'].rolling(7).mean()/dfc['end_new_cases_smoothed'].rolling(7).mean() 
        plot = go.Scatter(x=dfc.date,y=index, name=c)
        plots.append(plot)
    fig = go.Figure(data=plots, layout=go.Layout(barmode='group'))

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))

    fig.update_layout(
        height=600,
        title="Covid Import Index",
        xaxis_title="date",
        yaxis_title="covid import index in %"
    )
    return fig

plot_covid_import_index().show()

In [6]:
def plot_covid_import_index_heat_map():
    plots = []

    indices = []
    labels = []
    dates = df_in[df_in.end_name == 'belgium'].date.values
    n = len(dates)

    for c in eu_countries:
        dfc = df_in[df_in.end_name == c]
        index = 100*dfc['covid_flow'].rolling(7).mean()/dfc['end_new_cases_smoothed'].rolling(7).mean()
        
        if (len(index) == n):
            indices.append(index)
            labels.append(c)
 
    import plotly.graph_objects as go

    fig = go.Figure(data=go.Heatmap(
                       z=indices,
                       x=dates,
                       y=labels,
                       hoverongaps = False,zmin=0,zmax=5))
    fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
    fig.update_layout(template="plotly_white", height=700, margin=dict(l=0, r=0, t=40, b=0),title="covid import index")
    return fig
plot_covid_import_index_heat_map().show()

In [7]:
def plot_covid_import_concentration():
    plots = []

    for c in eu_countries:
        dfc = df_in[df_in.end_name == c]
        travel_counts = dfc['travel_counts'].rolling(7).mean()
        covid_flow = dfc['covid_flow'].rolling(7).mean()
        covid_per_million_in = 1000000*covid_flow/travel_counts
                                                       
        index = covid_per_million_in / dfc['end_new_cases_smoothed_per_million']
        plot = go.Scatter(x=dfc.date,y=index, name=c)
        plots.append(plot)
    fig = go.Figure(data=plots, layout=go.Layout(barmode='group'))

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))

    fig.update_layout(
        height=600,
        title="Covid Import Index",
        xaxis_title="date",
        yaxis_title="covid import index in %"
    )
    return fig

plot_covid_import_concentration().show()

In [8]:
def plot_covid_import_concentration_heat_map():
    plots = []

    indices = []
    labels = []
    dates = df_in[df_in.end_name == 'belgium'].date.values
    n = len(dates)

    for c in eu_countries:
        dfc = df_in[df_in.end_name == c]
   
        travel_counts = dfc['travel_counts'].rolling(7).mean()
        covid_flow = dfc['covid_flow'].rolling(7).mean()
        covid_per_million_in = 1000000*covid_flow/travel_counts
                                                       
        index = covid_per_million_in / dfc['end_new_cases_smoothed_per_million']
        
        if (len(index) == n):
            indices.append(index)
            labels.append(c)
 
    import plotly.graph_objects as go

    fig = go.Figure(data=go.Heatmap(
                       z=indices,
                       x=dates,
                       y=labels,
                       hoverongaps = False,zmin=0,zmax=2))
    fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
    fig.update_layout(template="plotly_white", height=700, margin=dict(l=0, r=0, t=40, b=0),title="covid concentration import index")
    return fig
plot_covid_import_concentration_heat_map().show()

In [60]:
df_countries = pd.read_csv('../static/csv/countries.csv')

eu_countries = ['austria','belgium' ,'croatia' ,'czech republic' ,'denmark',
 'estonia' ,'finland' ,'france' ,'germany', 'greece', 'iceland', 'ireland',
 'italy','lithuania' ,'luxembourg', 'netherlands', 'norway',
 'romania', 'serbia', 'spain', 'sweden', 'switzerland',
 'united kingdom', 'portugal', 'slovenia',  'hungary', 'poland',
 'bosnia and herzegovina', 'slovakia', 'bulgaria']


# filter eu countries
df_countries['Country'] = df_countries['Country'].str.lower()
df_countries = df_countries[df_countries['Country'].isin(eu_countries)]
latitude = df_countries.set_index(['Country']).to_dict(orient='dict')['Latitude']
longitude = df_countries.set_index(['Country']).to_dict(orient='dict')['Longitude']


import numpy as np
import pandas as pd
import plotly.graph_objects as go


def flow_arrows(fig,start_date,end_date):
    

    df_fb_ = df_fb[df_fb.date <= end_date]
    df_fb_ = df_fb_[df_fb_.date >= start_date]
    
    n = len(eu_countries)
    range_eu = range(len(eu_countries))
    mig = [[None for i in range_eu] for j in range_eu]

    import math
    df_mean_travels = df_fb_.groupby(['start_name', 'end_name']).agg({'travel_counts': 'mean'}).reset_index()
    dict_travels = df_mean_travels .set_index(['start_name','end_name']).to_dict(orient='dict')['travel_counts']
    
    for (c1,c2) in dict_travels.keys():
            flow_c1_c2 = dict_travels[(c1, c2)]
            fig.add_trace(go.Scattergeo(
                lat = [latitude[c1],latitude[c2]], 
                lon = [longitude[c1],longitude[c2]],
                mode = 'lines',
                line = dict(width = flow_c1_c2/10000, color = 'blue'),
                showlegend=False,
            ))
            
    fig.update_layout(
        title = 'Europe '+start_date+"-"+end_date,
    )



fig = go.Figure()
fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_countries['Longitude'],
        lat = df_countries['Latitude'],
        text = df_countries['Country'],
        textfont = {"color": 'black',
                    "family":'Times New Roman',
                    "size":14},
        textposition="top center",
        name = "Counry",
        showlegend=False,
        mode ="markers+text",
        marker = dict(
            size = 10,
            color = "black",
            line_color='black',
            line_width=0.5,
            sizemode = 'area')))
fig.update_layout(
        geo_scope='europe',
    )
fig.update_layout(template="plotly_white", height=850, margin=dict(l=0, r=0, t=40, b=0))

#flow_arrows(fig,'2020-03-07','2020-03-13')
#flow_arrows(fig,'2020-04-07','2020-04-13')
flow_arrows(fig,'2020-08-04','2020-08-11')
#flow_arrows(fig,'2020-11-15','2020-11-22')



fig.show()